In [1]:
import codecs
import re
import random
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score, accuracy_score

In [2]:
#将人工标注的数据转化成crf模型格式
def get_crf_data(line, words):
    indexs = []
    #for m in re.finditer(r'\[(.*?)\]',string):
    #    indexs += [m.start(0),m.end(0)]
    pre,flag,res = '',False,[]
    for i in range(len(line)):
        word = line[i]
        if word != '[' or word != ']':
            words.add(word)
        if word == '[':
            pre = word
            continue
        elif pre == '[':
            res.append([word,'B'])
            flag = True
        elif word == ']':
            res[-1][-1] = 'E'
            flag = False
        elif flag:
            res.append([word, 'M'])
        else:
            res.append([word, 'O'])
        pre = word
    return res

def process_file(path):
    res, max_len, words = [], 0, set()
    with codecs.open(path) as fin:
        for line in fin:
            line = line.strip()
            count = 0
            for word, label in get_crf_data(line, words):
                res.append(word + ' ' + label + '\n')
                count += 1
            if count > max_len: max_len = count
            res.append('\n')
    return max_len, words, res

def write_data(lines,out_path):
    with open(out_path,'w') as fout:
        for line in lines:
            fout.write(line)
def write_words(words, out_path):
    with codecs.open(out_path, 'w', encoding='gbk') as fout:
        for word in words:
            fout.write(word + '\n')

In [3]:
'''
max_len, words, res = process_file('data/bendibao.txt')
write_data(res, 'data/bendibao.crf')
print(max_len)
write_words(words, 'data/words.txt')
'''
#print(word2id)
#print(id2word)
#get_crf_data('杭州[去哪吃夜宵]？[杭州吃夜宵好去处]')

"\nmax_len, words, res = process_file('data/bendibao.txt')\nwrite_data(res, 'data/bendibao.crf')\nprint(max_len)\nwrite_words(words, 'data/words.txt')\n"

In [4]:
string = 'abc[bde]ioe[oeds]in'

In [5]:
re.findall(r'\[(.*?)\]',string)

['bde', 'oeds']

In [6]:
res = re.finditer(r'\[(.*?)\]',string)

In [7]:
for m in res:
    print(m.group(0),m.start(0),m.end(0))

[bde] 3 8
[oeds] 11 17


In [8]:
#tools，工具类
#将CRf数据格式进行处理，完成数据的填充，以及batch_size的数据集获取
class Tools:
    def __init__(self, seq_length):
        self.pad_word = '<PAD>'
        self.pad_tag = 'O'
        self.seq_length = seq_length
        self.words = []
        self.word2id = {}
        self.id2word = {}
        self.tag2id = {}
        
        self.get_tag2id()
        self.load_words('data/words.txt')
        self.get_word_dict()
    
    def get_tag2id(self):
        #self.tag2id = {'O':[1.,.0,.0,.0], 'B':[.0,1.,.0,.0], 'M':[.0,.0,1.,.0], 'E':[.0,.0,.0,1.]}
        self.tag2id = {'O':0, 'B':1, 'M':2, 'E':3}
    def load_words(self, path):
        
        with codecs.open(path, 'r', encoding='gbk') as fin:
            for line in fin:
                word = line.strip()
                if len(word) == 0: continue
                self.words.append(word)
                
    def get_word_dict(self):
        for index, word in enumerate(self.words, 1):
            self.word2id[word] = index
            self.id2word[index] = word
        self.word2id[self.pad_word] = 0
        self.id2word[0] = self.pad_word
    
    def padding_sequence(self, sequence, seq_length):
        train_list, tag_list = [], []
        #print(sequence)
        #print(seq_length)
        for i in range(seq_length):
            if i >= len(sequence):
                train_list.append(self.word2id[self.pad_word])
                tag_list.append(self.tag2id[self.pad_tag])
            else:
                try:
                    train_list.append(self.word2id[sequence[i][0]])
                    tag_list.append(self.tag2id[sequence[i][1]])
                except Exception as e:
                    print(str(e))
                    print(sequence)
        return train_list, tag_list
    
    def process_file(self, path):
        X, Y, sequence = [], [], []
        with codecs.open(path, 'r', encoding='gbk') as fin:
            for line in fin:
                if line == '\n':
                    sub_x, sub_y = self.padding_sequence(sequence, self.seq_length)
                    X.append(sub_x)
                    Y.append(sub_y)
                    sequence = []
                else:
                    items = line.strip().split(' ')
                    if len(items) > 1:
                        sequence.append(items)
            return np.array(X), np.array(Y)
    
    def next_batch(self,X, Y, batch_size=64):
        length = len(X)
        num_batch = length // batch_size
        indexs = list(range(length))
        random.shuffle(indexs)
        x_shuffle = X[indexs]
        y_shuffle = Y[indexs]
        
        for i in range(num_batch):
            start = i * batch_size
            end = min((i+1) * batch_size, length)
            yield x_shuffle[start:end], y_shuffle[start:end]

In [9]:
tools = Tools(37)
X,Y = tools.process_file('data/bendibao.train')
X_val, Y_val = tools.process_file('data/bendibao.test')
print(len(tools.word2id))

1745


In [10]:
print(len(X), len(X_val))
print(X_val,Y_val)
#print(tools.id2word)

4667 1444
[[ 706 1470 1641 ...    0    0    0]
 [ 810  236   53 ...    0    0    0]
 [ 126 1248  384 ...    0    0    0]
 ...
 [  26 1162  706 ...    0    0    0]
 [ 706 1470 1737 ...    0    0    0]
 [1414 1270 1091 ...    0    0    0]] [[1 2 2 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 2 2 ... 0 0 0]
 [1 2 2 ... 0 0 0]
 [1 2 2 ... 0 0 0]]


In [11]:
'''
count = 0
for x, y in tools.next_batch(X, Y, 64):
    count += 1
    if count == 2: break
    print(x)
    print(y)
'''

'\ncount = 0\nfor x, y in tools.next_batch(X, Y, 64):\n    count += 1\n    if count == 2: break\n    print(x)\n    print(y)\n'

In [12]:
#构建lstm 网络
class lstm_model(object):
    def __init__(self):
        self.batch_size = 64
        self.seq_length = 37
        self.num_classes = 4
        self.lr = 0.02
        self.lr_decay = 0.9
        self.dropout_keep_pro = 0.8
        self.num_epoch = 20
        self.hidden_dim = 128
        self.num_layer = 2
        self.embedding_dim = 64
        self.vocab_size = 1745
        
        self.run()
        
    def run(self):
        #inpux
        tf.reset_default_graph()
        self.input_x = tf.placeholder(dtype=tf.int32, shape=[None, self.seq_length], name='input_x')
        self.input_y = tf.placeholder(dtype=tf.int32, shape=[None, self.seq_length], name='input_y')
        self.sequence_lengths = tf.constant([self.seq_length] * self.batch_size, dtype=tf.int32)
        self.dropout_keep_pro = tf.placeholder(dtype=tf.float32, name='keep_pro')
        
        #input_y 变形
        #self.labels = tf.reshape(self.input_y, [-1])
        
        #lstm
        def lstm_cell():
            return tf.contrib.rnn.LSTMCell(self.hidden_dim)
        
        def dropout():
            cell = lstm_cell()
            return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=self.dropout_keep_pro)
        
        def get_weight(shape):
            return tf.Variable(tf.random_normal(shape=shape, stddev=0.1))
        
        #embedding
        with tf.name_scope('embedding'):
            embedding = tf.get_variable('embedding', shape=[self.vocab_size, self.embedding_dim], dtype=tf.float32)
            embedding_inputs = tf.nn.embedding_lookup(embedding, self.input_x)
    
        with tf.name_scope('bi_lstm'):
            #cells = [dropout() for i in range(self.num_layer)]
            #rnn_cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
            #fw_cell, bw_cell = lstm_cell(), lstm_cell
            # lstm模型　正方向传播的RNN
            lstm_fw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.hidden_dim, forget_bias=1.0)
            # 反方向传播的RNN
            lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(self.hidden_dim, forget_bias=1.0)
            #print(fw_cell)
            (fw_outputs, bw_outputs),_ = tf.nn.bidirectional_dynamic_rnn(lstm_fw_cell, lstm_bw_cell, embedding_inputs, dtype=tf.float32)
            outputs = tf.concat([fw_outputs, bw_outputs], axis= 2)
            
            #outputs,_ = tf.nn.dynamic_rnn(cell=rnn_cell, inputs=embedding_inputs, dtype=tf.float32)
            print(embedding_inputs.shape)
            print(outputs.shape)
            print(outputs[:,-1,:].shape)
            
            #改变形状
            outputs = tf.reshape(outputs, [-1, 2 * self.hidden_dim])
        
        with tf.name_scope('full_layer'):
            weight1 = get_weight(shape=[2 * self.hidden_dim, self.hidden_dim])
            biases = tf.Variable(tf.constant(0.1,dtype=tf.float32, shape = [self.hidden_dim]))
            fc1 = tf.matmul(outputs, weight1) + biases
            fc1 = tf.nn.dropout(fc1, keep_prob=self.dropout_keep_pro)
            fc1 = tf.nn.relu(fc1)
            
            weigth2 = get_weight(shape=[self.hidden_dim, self.num_classes])
            biases2 = tf.Variable(tf.constant(0.1,dtype=tf.float32, shape = [self.num_classes]))
            self.logits = tf.matmul(fc1, weigth2) + biases2
            self.logits = tf.reshape(self.logits, [-1,self.seq_length, self.num_classes])
        
        with tf.name_scope('crf_layer'):
            print(self.logits.shape, self.input_y.shape)
            # crf 得到维特比矩阵 
            log_likelihood, self.transition_params = tf.contrib.crf.crf_log_likelihood(\
                            self.logits, self.input_y, self.sequence_lengths)
            
        with tf.name_scope('train_step'):
            #print('labels',self.labels.shape)
            #print('logits',self.logits.shape)
            #cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.labels, logits=self.logits)
            #tf.nn.sparse_softmax_cross_entropy_with_logits(
            loss = tf.reduce_mean(-log_likelihood)
            self.train_step = tf.train.GradientDescentOptimizer(self.lr).minimize(loss)
        
        '''
        with tf.name_scope('accuracy'):
            self.y_true = tf.reshape(self.input_y, [-1])
            correct_pre = tf.equal(self.y_pre, self.y_true)
            self.accuracy = tf.reduce_mean(tf.cast(correct_pre,dtype=tf.float32))
        '''

In [62]:
class Main(object):
    def __init__(self, X, Y, X_val, Y_val):
        self.model = lstm_model()
        self.save_path = 'model/bi_lstm_crf_model/model.ckpt'
        self.X = X
        self.Y = Y
        self.X_val = X_val
        self.Y_val = Y_val
        self.save_epoch = 20 #最佳结果后再训练10轮，如果结果不能更好就停止训练
        
    def get_y_pre(self, sess, x, y):
        
        logits, transition_params = sess.run([self.model.logits, self.model.transition_params],
                          {self.model.input_x:x,self.model.input_y:y, self.model.dropout_keep_pro:1.0})
      
        y_pre = []
        for logit in logits:
            sub_y, _ = tf.contrib.crf.viterbi_decode(logit, transition_params)
            y_pre += list(sub_y)
        return y_pre
    
    def evaluate(self, sess, x, y):
        y_true = y.reshape([-1])
        y_pre = get_y_pre(sess, x, y)
        
        score = f1_score(y_true, y_pre, [0.,1.,2.,3.], average=None)
        acc = accuracy_score(y_true, y_pre)
        score = np.insert(score, 0, acc)
        return score
    
    def evaluate2(self, y_true, y_pre):
        score = f1_score(y_true, y_pre, [0.,1.,2.,3.], average=None)
        acc = accuracy_score(y_true, y_pre)
        score = np.insert(score, 0, acc)
        return score
    
    #训练模型
    def train(self, num_epochs):
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()
        max_train_f1_score, max_test_f1_score, count = 0,0,0
        
        with tf.Session() as sess:
            sess.run(init)
            print(X.shape)
            print(Y.shape)
            for epoch in range(num_epochs):
                for xs,ys in tools.next_batch(self.X, self.Y):
                    sess.run(self.model.train_step, feed_dict = \
                            {self.model.input_x:xs, self.model.input_y:ys, self.model.dropout_keep_pro:0.8})
                    
                self.model.lr *= self.model.lr_decay
                #train_acc = sess.run(self.model.accuracy, feed_dict = \
                #    {self.model.input_x:self.X, self.model.input_y:self.Y, self.model.dropout_keep_pro:1.0})
                #test_acc = sess.run(self.model.accuracy, feed_dict = \
                #    {self.model.input_x:self.X_val, self.model.input_y:self.Y_val, self.model.dropout_keep_pro:1.0})
                train_score = self.evaluate(sess, self.X, self.Y)
                test_score = self.evaluate(sess, self.X_val, self.Y_val)
                train_f1_score = np.mean(train_score[1:])
                test_f1_score = np.mean(test_score[1:])
                
                train_info = ', Training Accuracy:%.3f | c0 f1_score:%.3f, c1 f1_score:%.3f, c2 f1_score:%.3f, c3 f1_score:%0.3f' % tuple(train_score)
                test_info = ', Testing  Accuracy:%.3f | c0 f1_score:%.3f, c1 f1_score:%.3f, c2 f1_score:%.3f, c3 f1_score:%0.3f' % tuple(test_score)
                print('Iter ' + str(epoch) +  train_info)
                print('Iter ' + str(epoch) +  test_info)
                if train_f1_score > max_train_f1_score and test_f1_score > max_test_f1_score:
                    print('update')
                    max_train_f1_score = train_f1_score
                    max_test_f1_score = train_f1_score
                    saver.save(sess, self.save_path)
                    count = 0
                else:
                    count += 1
                    
                if count > self.save_epoch:
                    break
                print('count:', count)
            
    def test(self, id2word):
        #saver = tf.train.import_meta_graph(self.save_path + '.meta')  #这个不行
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()
        with tf.Session() as session:
            session.run(init)
            
            saver.restore(session, self.save_path)
            y_pre = self.get_y_pre(session, self.X_val, self.Y_val)
            y_true = self.Y_val.reshape([-1])
            count = 0
            for i in range(len(X_val)):
                line, labels = X_val[i], Y_val[i]
                for j in range(len(line)):
                    print(id2word[line[j]], labels[j], y_pre[count])
                    count += 1
            val_score = self.evaluate2(y_true, y_pre)
            val_info = 'Testing  Accuracy:%.3f | c0 f1_score:%.3f, c1 f1_score:%.3f, c2 f1_score:%.3f, c3 f1_score:%0.3f' % tuple(val_score)
            print(val_info)

In [63]:
main = Main(X, Y, X_val, Y_val)
#main.train(300)

(?, 37, 64)
(?, 37, 256)
(?, 256)
(?, 37, 4) (?, 37)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [64]:
main = Main(X, Y, X_val, Y_val)
main.test(tools.id2word)

(?, 37, 64)
(?, 37, 256)
(?, 256)
(?, 37, 4) (?, 37)
INFO:tensorflow:Restoring parameters from model/bi_lstm_crf_model/model.ckpt


/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


北 1 1
京 2 2
朝 2 2
阳 2 2
浪 2 2
漫 2 2
约 2 2
会 2 2
好 2 2
去 2 2
处 3 3
推 0 0
荐 0 0
两 0 0
个 0 0
人 0 0
最 0 0
美 0 0
的 0 0
时 0 0
光 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
2 0 0
0 0 0
1 0 0
4 0 0
暑 0 0
期 0 0
北 1 1
京 2 2
周 2 2
边 2 2
亲 2 2
水 2 2
游 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
佛 0 0
山 0 0
夏 0 0
季 0 0
哪 0 0
里 0 0
好 0 0
玩 0 0
？ 0 0
佛 1 1
山 2 2
周 2 2
边 2 2
避 2 2
暑 2 2
好 2 2
去 2 2
处 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
北 1 1
京 2 2
五 2 2
一 2 2
京 2 2
郊 2 2
游 2 2
观 2 2
鸟 2 2
地 3 3
推 0 0
荐 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 

<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
南 1 1
京 2 2
赏 2 2
桂 2 2
花 2 2
线 2 2
路 3 3
推 0 0
荐 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
上 1 1
海 2 2
最 2 2
具 2 2
情 2 2
调 2 2
的 2 2
6 2 2
个 2 2
公 2 2
园 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
端 0 0
午 0 0
节 0 0
去 0 0
哪 0 0
约 0 0
会 0 0
好 0 0
？ 0 0
广 0 0
东 0 0
2 0 0
0 0 0
1 0 0
5 0 0
端 1 1
午 2 2
节 2 2
情 2 2
侣 2 2
游 2 2
好 2 2
去 2 2
处 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
2 0 0
0 0 0
1 0 0
6 0 0
年 0 0
重 1 1


<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
重 1 1
庆 2 2
主 2 2
城 2 2
适 2 2
合 2 2
带 2 2
小 2 2
孩 2 2
子 2 2
去 2 2
玩 2 2
的 2 2
地 2 2
方 3 3
有 0 0
哪 0 0
些 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
2 0 0
0 0 0
1 0 0
4 0 0
暑 1 0
假 2 0
北 2 1
京 2 2
旅 2 2
游 2 2
好 2 2
去 2 2
处 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
2 0 0
0 0 0
1 0 0
6 0 0
年 0 0
重 1 1
庆 2 2
春 2 2
节 2 2
泡 2 2
温 2 2
泉 2 2
的 2 2
地 2 2
方 3 3
推 0 0
荐 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<P

...

2 2 2
月 2 2
南 2 2
京 2 2
好 2 2
玩 2 2
的 2 2
地 2 2
方 3 3
四 0 0
个 0 0
周 0 0
末 0 0
这 0 0
样 0 0
安 0 0
排 0 0
最 0 0
好 0 0
不 0 0
过 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
京 1 1
郊 2 2
采 2 2
摘 2 2
园 2 2
好 2 2
地 2 2
方 3 3
周 0 0
末 0 0
当 0 0
回 0 0
农 0 0
民 0 0
找 0 0
找 0 0
乐 0 0
( 0 0
图 0 0
) 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
重 1 1
庆 2 2
冬 2 2
季 2 2
好 2 2
耍 2 2
地 2 3
推 2 0
荐 3 0
： 0 0
赏 0 0
红 0 0
叶 0 0
泡 0 0
温 0 0
泉 0 0
活 0 0
动 0 0
多 0 0
多 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
上 1 1
海 2 2
周 2 2
边 2 2
最 2 2
值 2 2
得 2 2
去 2 2
的 2 2
8 2 2
大 2 2
亲 2 2
子 2 2
酒 2 2
店 3 2
爸 0 2
爸 0 2
去 0 2
哪 0 2
儿 0 3
<PAD> 0 0
<PAD> 0 0
<PAD

昌 2 2
平 2 2
冬 2 2
季 2 2
游 2 2
玩 2 2
推 2 2
荐 3 3
( 0 0
滑 0 0
雪 0 0
+ 0 0
温 0 0
泉 0 0
+ 0 0
摘 0 0
草 0 0
莓 0 0
) 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
中 0 0
国 0 0
9 0 0
月 0 0
份 0 0
去 0 0
哪 0 0
里 0 0
旅 0 0
游 0 0
？ 0 0
2 0 0
0 0 0
1 0 0
7 0 0
年 0 0
9 0 0
月 0 0
份 0 0
国 1 1
内 2 2
旅 2 2
游 2 2
好 2 2
去 2 2
处 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
2 0 0
0 0 0
1 0 0
4 0 0
北 1 1
京 2 2
周 2 2
边 2 2
游 3 2
山 0 2
玩 0 2
水 0 2
路 0 2
线 0 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
千 0 0
岛 0 0
湖 0 0
有 0 0
什 0 0
么 0 0
好 0 0
玩 0 0
的 0 0
？ 0 0
千 1 1
岛 2 2
湖 2 2
周 2 2
末 2 2
二 2 2
日 2 2
游 3 3
全 0 0
攻 0 0
略 0 0
<PAD> 0 0
<PAD> 0 0


婚 2 2
纱 2 2
照 2 2
的 2 2
小 2 2
清 2 2
新 2 2
外 2 2
景 2 2
地 3 3
推 0 0
荐 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
天 1 1
津 2 2
红 2 2
色 2 2
主 2 2
题 2 2
旅 2 2
游 3 3
攻 0 0
略 0 0
之 0 0
名 0 0
人 0 0
故 0 0
居 0 0
线 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
广 1 1
州 2 2
周 2 2
末 2 2
免 2 2
费 2 2
玩 2 2
乐 2 2
好 2 2
去 2 2
处 3 3
（ 0 0
2 0 0
月 0 0
4 0 0
日 0 0
- 0 0
2 0 0
月 0 0
1 0 0
0 0 0
日 0 0
） 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
昆 1 1
明 2 2
周 2 2
边 2 2
9 2 2
条 2 2
自 2 2
驾 2 2
路 2 2
线 3 3
推 0 0
荐 0 0
（ 0 0
详 0 0
细 0 0
） 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 

<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
广 1 1
州 2 2
附 2 2
近 2 2
泡 2 2
温 2 2
泉 2 2
好 2 2
去 2 2
处 3 3
汇 0 0
总 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
天 1 1
津 2 2
春 2 2
节 2 2
自 2 2
驾 2 2
游 3 3
以 0 0
后 0 0
再 0 0
也 0 0
不 0 0
用 0 0
下 0 0
江 0 0
南 0 0
（ 0 0
附 0 0
自 0 0
驾 0 0
攻 0 0
略 0 0
） 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
昌 1 1
平 2 2
骑 2 2
行 2 2
路 2 2
线 3 3
推 0 0
荐 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<P

处 3 3
上 1 1
海 2 2
周 2 2
边 2 2
避 2 2
暑 2 2
胜 2 2
地 3 3
推 0 0
荐 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
北 1 1
京 2 2
郊 2 2
区 2 2
适 2 2
合 2 2
聚 2 2
会 2 2
的 2 2
地 2 2
方 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
2 0 0
0 0 0
1 0 0
5 0 0
六 1 1
一 2 2
儿 2 2
童 2 2
节 2 2
广 2 2
州 2 2
自 2 2
驾 2 2
游 2 2
好 2 2
去 2 2
处 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
十 1 1
月 2 2
北 2 2
京 2 2
自 2 2
驾 2 2
游 2 2
景 2 2
点 3 3
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<PAD> 0 0
<P